In [1]:
# Import required modules
import numpy as np
import math
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from bs4.element import Tag
import pprint
import traceback
import logging

In [2]:
# Create a variable with the URL to this tutorial
url = 'http://www.englishforum.ch/forum.php'
# Scrape the HTML at the url
r = requests.get(url)
# Turn the HTML into a Beautiful Soup object
soup = BeautifulSoup(r.text, 'lxml')

In [3]:
# Create four variables to score the scraped data in
dataframes={}
for link in soup.find_all("strong")[1:]:
    for linkhref in soup.find_all("a"):
        if(link in linkhref and not linkhref['href'].endswith('.html') and link.contents[0] not in dataframes):
            dataframes[link.contents[0]]=linkhref['href']
dataframes


{'Business & entrepreneur': 'http://www.englishforum.ch/business-entrepreneur/',
 'Commercial events': 'http://www.englishforum.ch/commercial-events/',
 'Complaints corner': 'http://www.englishforum.ch/complaints-corner/',
 'Concerts': 'http://www.englishforum.ch/concerts/',
 'Daily life': 'http://www.englishforum.ch/daily-life/',
 'Education': 'http://www.englishforum.ch/education/',
 'Employment': 'http://www.englishforum.ch/employment/',
 'Entertainment & dining': 'http://www.englishforum.ch/entertainment-dining/',
 'Family matters/health': 'http://www.englishforum.ch/family-matters-health/',
 'Finance/banking/taxation': 'http://www.englishforum.ch/finance-banking-taxation/',
 'Food and drink': 'http://www.englishforum.ch/food-drink/',
 'Housing in general': 'http://www.englishforum.ch/housing-general/',
 'Insurance': 'http://www.englishforum.ch/insurance/',
 'Introductions': 'http://www.englishforum.ch/introductions/',
 'Language corner': 'http://www.englishforum.ch/language-corner

# Fetch the info from urls

In [33]:
def comments(numPag,thread):
    # Scrape the HTML at the url
    response = requests.get(thread,headers={'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'})
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(response.text, 'lxml')
    comments={}
    inception={}
    date=''
    first=[]
    user_and_rep=[]
    for link in soup.find_all("div",{"align":"center"}):
        for link2 in link.find_all("div",{"align":"left"}):
            for link3 in link2.find_all("tr"):
                for link4 in link3.find_all("td",{"width":"99%"}):
                    for link5 in link4.find_all("div"):
                        if('id' in link5.attrs and 'posts' in link5['id']):
                            link6=link5.find('table')
#                             print(link6)

                            try:
                                link7=link6.find('td').find_all('div')
                                if(len(link7)==0):
                                    date='null'
                                else:
                                    date=link7[-1].contents[-1].strip(' \n\r\t')
                            except IndexError as  err:
                                print(link6)
                                print(thread)
                                logging.error(traceback.format_exc())
                                
                            if(numPag==0 and 'null' not in date):
                                for ind,link7 in enumerate(link6.find_all('div')):
#                                 print(link7.contents[0])
                                    if('href' not in link7 and len(link7.contents)>0):
#                                         print('aaa',ind,type(link7.contents[0]),link7.contents[0])
                                        if(type(link7.contents[0])==Tag):
#                                         link7=link7.find('span')
                                            if(len(link7.attrs)==0):
#                                             print('aaa',ind,type(link7.contents[0]),link7.attrs,link7.contents[0])
                                                    user_and_rep=str(link7.contents[0]).partition('alt=')[2]
                                                    user_and_rep=user_and_rep.partition(user_and_rep[0]+' border')[0]
                                                    if('has' in user_and_rep):
                                                        user_and_rep=user_and_rep.partition(' has ')
                                                    elif('is' in user_and_rep):
                                                        user_and_rep=user_and_rep.partition(' is ')
                                                    user_and_rep=[user_and_rep[0],user_and_rep[2]]
                                               
#                                         print(user_and_rep)
                                        else:
                                            stat=link7.contents[0].strip('\n\r\t')
#                                         if(stat!=' ' and ord(stat)!=160):
                                            if(len(stat)>1 and ord(stat[0])!=160):
#                                             print(link7)
                                                first.append(stat)
#                             if(len(user_and_rep)>0):
                        if('id' in link5.attrs and link5['id'].startswith('post_message')):
#                             print(link5.attrs,link5.contents)
                            contents=''
                            for content in link5.contents:
                                if type(content) is not Tag:
                                    contents+=' '+content
#                                 comments[link5['id']]=content
                                elif(str(content).startswith('<font')):
                                    contents+=' '+re.sub(r'<.*?>', '', str(content))
                                
                            comments[link5['id']]=contents
    retcomments=''
    for ind,i in enumerate(sorted(comments)):
#          print(comments[i])
        if(numPag==0):
#             print(user_and_rep,'len',len(first),first)
            if(len(first)==6):
                first.append(comments[i])
            else:
                first[-1]=comments[i]
        retcomments+=comments[i]
#     if(numPag==0):
#         try:
#             inception['user']=user_and_rep[0]
#             inception['date']=date
#             inception['reputation']=user_and_rep[1]
#             inception['exp']=first[0]
#             inception['since']=first[1]
#             inception['location']=first[2]
#             inception['posts']=first[3]
#             inception['thanked']=first[4]
#             inception['groaned']=first[5]
#             inception['post']=first[6]
#         except IndexError as err:
#             print(len(first),first)
#             print(user_and_rep,'len',len(first),first)
#             print('IndexError:', err)
        try:
            if(numPag==0 and len(first)==7 and 'null' not in date):
                inception['user']=user_and_rep[0][1:]
                inception['date']=date
                inception['reputation']=user_and_rep[1]
                inception['exp']=first[0]
                inception['since']=first[1]
                inception['location']=first[2]
                inception['posts']=first[3]
                inception['groaned']=first[4]
                inception['thanked']=first[5]
                inception['post']=first[6]
            elif(numPag==0):
                inception['user']='guest'
                inception['date']='null'
                inception['reputation']='null'
                inception['exp']='null'
                inception['since']='null'
                inception['location']='null'
                inception['posts']='null'
                inception['thanked']='null'
                inception['groaned']='null'
                inception['post']=first[-1]
        except Exception as e:
            logging.error(traceback.format_exc())
            print(thread)
            if(numPag==0):
                inception['user']='guest'
                inception['date']='null'
                inception['reputation']='null'
                inception['exp']='null'
                inception['since']='null'
                inception['location']='null'
                inception['posts']='null'
                inception['thanked']='null'
                inception['groaned']='null'
                inception['post']=first[-1]
#             print(len(first),first)
#             print(user_and_rep,'len',len(first),first)
#             print('IndexError:', err)
    return [retcomments,inception]

def page(url,index):
#     print(url,index)
    # Scrape the HTML at the url
    r = requests.get(url)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    subtopic={}
    for link in soup.find_all("table",id="threadslist"):
#         print(link)
        index_aux=index
        dash=0
        for replies in link.find_all("td",class_="alt1"):
            if(replies.contents[0]!=' ' and type(replies.contents[0])!=Tag):
                if(replies.contents[0]=='-'):
                    dash+=1
                subtopic[index_aux]=[]
                subtopic[index_aux].append(replies.contents[0])
# #                 print('rep',index_aux,replies.contents[0])

                index_aux+=1
    
#         print('Replies found:',index_aux)
        index_aux=index
        insertdash=0
        # \xa0 Unicode --> ascii 160
        for views in link.find_all("td",class_="alt2"):
            if('align' in views.attrs and views.contents[0]!=' ' and ord(views.contents[0][0])!=160 and type(views.contents[0])!=Tag):
                if(views.contents[0]=='-'):
                    if(insertdash==0):
                        insertdash=1
                    else:
                        insertdash=0
                        subtopic[index_aux].append(views.contents[0])
                        index_aux+=1
                else:
                    subtopic[index_aux].append(views.contents[0])
                    index_aux+=1    
#         print('Views found:',index_aux)
        for thread in link.find_all("a"):
            if('id' in thread.attrs):
#                 print('hrefs',index,link1.contents[0])
                    subtopic[index].append(thread['href'])
                    subtopic[index].append(thread.contents[0])
                    index+=1
        
#         print('Threads found:',index)
    return [subtopic,index]
    
    
def findAllPages(rootHTML):
    # Scrape the HTML at the url
    r = requests.get(rootHTML)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    lastUrl=''
    pages=[]
    pages.append(rootHTML)
    for link in soup.find_all("body"):
        for link1 in link.find_all("a"):
            if('title' in link1.attrs and 'Last' in link1['title']):
                lastUrl=link1['href']
#                 print(link1['href'])
                break
    if(lastUrl==''):#no Last page link, we fetch last index page
        links=set()
        for link in soup.find_all("body"):
            for link1 in link.find_all("a"):
                if('title' in link1.attrs and 'Show results' in link1['title']):
                    links.add(link1['href'])
#                     print(link1)
        if(links!=set() and rootHTML!=sorted(links)[-1]):
            lastUrl=sorted(links)[-1]
    if(lastUrl!=''):
        if(rootHTML.endswith('.html')):
            limit=re.findall(r'\d+',lastUrl)[-1]
            for i in range(2,int(limit)):
                pages.append(re.sub(r'.html','',rootHTML)+"-"+str(i)+".html")
        else:
            limit=re.findall(r'\d+',lastUrl)[0]
            for i in range(2,int(limit)):
                pages.append(rootHTML+"index"+str(i)+".html")
        pages.append(lastUrl)
        return pages
    return pages
    
def createDF(url):
#     print(url)
    index=0
    allsubtopics={}
    nextPages=findAllPages(url)
    for url in nextPages:
#         print('urlllll',url)
        result=page(url,index)
        allsubtopics.update(result[0])
        index=result[1]
    #         print('fin',subtopic)
    replies=[]
    views=[]
    hrefs=[]
    threads=[]
    first_posts=[]
    print("For this tipic: %s there are %d subtopics" %(url,len(allsubtopics)))
    for value in allsubtopics.values():
        replies.append(value[0])
        views.append(value[1])
        hrefs.append(value[2])
        threads.append(value[3])
    posts=[]
    porcentaje=0
    for indexhref,href in enumerate(hrefs):
        nextPages=findAllPages(href)
        threadPosts=[]
        for numPag,pag in enumerate(nextPages):
#             if(numPag==0):
#                 print('pagina inicial del thread:'pag)
#             threadPosts.append(comments(pag))
            comments_call=comments(numPag,pag)
            threadPosts.append(comments_call[0])
            if(numPag==0):
                first_posts.append(comments_call[1])
        posts.append(''.join(threadPosts))
        if((((indexhref*100)/len(hrefs)))>porcentaje):
            print('Percentage of thread: ',int((indexhref*100)/len(hrefs)))
            porcentaje=porcentaje+20;
    date=[]
    exp=[]
    groaned=[]
    location=[]
    post=[]
    user_posts=[]
    reputation=[]
    since=[]
    thanked=[]
    user=[]
#     example of element first_posts
#   {'date': '19.12.2014',
#   'exp': 'Newbie',
#   'groaned': 'Thanked 0 Times in 0 Posts',
#   'location': 'Location: Basel',
#   'post': "Hi all. new to this site, i've been in basel for about 6 months now and was wondering about where and how i can buy e-cigarettes. i've heard they help people quit smoking and i read they are much cheaper too.",
#   'posts': 'Posts: 6',
#   'reputation': 'has made some interesting contributions',
#   'since': 'Join Date: Dec 2014',
#   'thanked': 'Groaned at 1 Time in 1 Post',
#   'user': 'WisePotato'}]
    for index,stat in enumerate(first_posts):
        if(len(stat)<9):
            date.append('null')
            exp.append('null')
            groaned.append('null')
            location.append('null')
            post.append('null')
            user_posts.append('null')
            reputation.append('null')
            since.append('null')
            thanked.append('null')
            user.append('null')
#             print('print stats:',stat)
#             print('print index:',index)
#             print('hrefs',len(hrefs),'firsts',len(first_posts))
#             print(index,hrefs[index])
        else:
            date.append(stat['date'])
            exp.append(stat['exp'])
            groaned.append(stat['groaned'])
            location.append(stat['location'])
            post.append(stat['post'])
            user_posts.append(stat['posts'])
            reputation.append(stat['reputation'])
            since.append(stat['since'])
            thanked.append(stat['thanked'])
            user.append(stat['user'])
#     df = pd.DataFrame({'replies': replies, 'views': views, 'hrefs': hrefs, 'threads' : threads, 'posts': posts})
#     firstDF=pd.DataFrame({'user':user,'thread':threads,'date':date,'exp':exp,'groaned':groaned,'location':location,'post':post,'user_posts':user_posts,'reputation':reputation,'since':since,'thanked':thanked})
    joindf=pd.DataFrame({'thread':threads,'user':user,'views': views,'replies': replies,'location':location,'date':date,'post':posts,'user_posts':user_posts,'since':since,'exp':exp,'thanked':thanked,'groaned':groaned,'reputation':reputation})
    return joindf

# Save the data into csv folder and remove dataframe

In [8]:
def saveData(dataframe,i):
    dataframe.to_csv("csv/"+i.replace("/","")+".csv", sep='\t', encoding='utf-8',)
    del dataframe

# Dataframes we'd like to fetch

In [9]:
# for i in dataframes:
for index,i in enumerate(sorted(dataframes)):
    print(index,i,dataframes[i])

0 Business & entrepreneur http://www.englishforum.ch/business-entrepreneur/
1 Commercial events http://www.englishforum.ch/commercial-events/
2 Complaints corner http://www.englishforum.ch/complaints-corner/
3 Concerts http://www.englishforum.ch/concerts/
4 Daily life http://www.englishforum.ch/daily-life/
5 Education http://www.englishforum.ch/education/
6 Employment http://www.englishforum.ch/employment/
7 Entertainment & dining http://www.englishforum.ch/entertainment-dining/
8 Family matters/health http://www.englishforum.ch/family-matters-health/
9 Finance/banking/taxation http://www.englishforum.ch/finance-banking-taxation/
10 Food and drink http://www.englishforum.ch/food-drink/
11 Housing in general http://www.englishforum.ch/housing-general/
12 Insurance http://www.englishforum.ch/insurance/
13 Introductions http://www.englishforum.ch/introductions/
14 Language corner http://www.englishforum.ch/language-corner/
15 Leaving Switzerland http://www.englishforum.ch/leaving-switzerl

# In order to read from csv files

In [10]:
def readData(filename):
    if( not filename.endswith('.csv')):
        filename+='.csv'
    df = pd.read_csv('csv/'+filename, header=0,sep='\t',index_col=0)
    return df

# CSV files we have got so far

In [11]:
import os
for index,i in enumerate(sorted(os.listdir("csv/"))):
    print(index,i)

0 Business & entrepreneur.csv
1 Business & entrepreneurAuthors.csv
2 Commercial events.csv
3 Commercial eventsAuthors.csv
4 Complaints corner.csv
5 Complaints cornerAuthors.csv
6 Concerts.csv
7 ConcertsAuthors.csv
8 Daily life.csv
9 Daily lifeAuthors.csv
10 Education.csv
11 EducationAuthors.csv
12 Employment.csv
13 EmploymentAuthors.csv
14 Entertainment & dining.csv
15 Entertainment & diningAuthors.csv
16 Family mattershealth.csv
17 Family mattershealthAuthors.csv
18 Financebankingtaxation.csv
19 FinancebankingtaxationAuthors.csv
20 Food and drink.csv
21 Food and drinkAuthors.csv
22 Insurance.csv
23 InsuranceAuthors.csv
24 Introductions.csv
25 IntroductionsAuthors.csv
26 Language corner.csv
27 Language cornerAuthors.csv
28 Leaving Switzerland.csv
29 Leaving SwitzerlandAuthors.csv
30 Othergeneral.csv
31 OthergeneralAuthors.csv
32 Permitsvisasgovernment.csv
33 PermitsvisasgovernmentAuthors.csv
34 Pet corner.csv
35 Pet cornerAuthors.csv
36 Social events.csv
37 Social eventsAuthors.csv
38 

# In case we'd like to update the files, let's declare the variables

In [12]:
business_enterpreteur=None
complaints_corner=None
concerts=None
daily_life=None
employment=None
entertainment_dining=None
family_matters_health=None
finance_banking_taxation=None
food_drink=None
housing=None
insurance=None
introductions=None
language_corner=None
leaving_switzerland=None
market_place=None
off_topic=None
other_general=None
permits_visas_gov=None
pet_corner=None
sports_wellness=None
support=None
swiss_politics_news=None
transportation_driving=None
travel_free_time=None
tv_internet_telephone=None
# NON FETCHED BUT YET TO COME
commercial_events=None
education=None
social_events=None
swiss_news=None

# Let's download them

Download all the data from the forum can take around 1-2 days and some times we just want to update part of the data. 
The cell below allow us to choose which topics (of the forum) we would like to download. 
By commenting lines you will select he topics you want to download.

In [34]:
for index,i in enumerate(sorted(dataframes)):
    print(index, dataframes[i])
#     df=createDF(dataframes[i][0])
#     saveData(df)
    if index==0: skip=1          #  business_enterpreteur
    elif index==1: skip=1        #  commercial-events
    elif index==2: skip=1        #  complaints-corner
#     elif index==3: skip=1        #  concerts 
    elif index==4: skip=1        #  daily-life
    elif index==5: skip=1        #  education
    elif index==6: skip=1        #  employment
    elif index==7: skip=1        #  entertainment-dining
    elif index==8: skip=1        #  family-matters-health
    elif index==9: skip=1        #  finance-banking-taxation
    elif index==10: skip=1       #  food-drink
    elif index==11: skip=0       #  housing-general/
    elif index==12: skip=1       #  insurance
    elif index==13: skip=1       #  introductions
    elif index==14: skip=1       #  language_corner
    elif index==15: skip=1       #  leaving_switzerland
    elif index==16: skip=0       #  market_place
    elif index==17: skip=0       #  off_topic
    elif index==18: skip=1       #  other_general
    elif index==19: skip=1       #  permits_visas_gov
    elif index==20: skip=1       #  pet_corner
    elif index==21: skip=1       #  social_events
    elif index==22: skip=1       #  sports_wellness
    elif index==23: skip=0       #  support
    elif index==24: skip=1       #  swiss_news
    elif index==25: skip=1       #  swiss_politics_news
    elif index==26: skip=1       #  tv_internet_telephone
    elif index==27: skip=1       #  transportation_driving
    elif index==28: skip=1
    else:     
        try:
            createDFRet=createDF(dataframes[i])
            createDFRet['topic']=i
            saveData(createDFRet,(i+"Authors"))
        except Exception as e:
            logging.error(traceback.format_exc())

0 http://www.englishforum.ch/business-entrepreneur/
1 http://www.englishforum.ch/commercial-events/
2 http://www.englishforum.ch/complaints-corner/
3 http://www.englishforum.ch/concerts/
For this tipic: http://www.englishforum.ch/concerts/index29.html there are 848 subtopics
Percentage of thread:  0
Percentage of thread:  20
Percentage of thread:  40
Percentage of thread:  60
Percentage of thread:  80
4 http://www.englishforum.ch/daily-life/
5 http://www.englishforum.ch/education/
6 http://www.englishforum.ch/employment/
7 http://www.englishforum.ch/entertainment-dining/
8 http://www.englishforum.ch/family-matters-health/
9 http://www.englishforum.ch/finance-banking-taxation/
10 http://www.englishforum.ch/food-drink/
11 http://www.englishforum.ch/housing-general/
12 http://www.englishforum.ch/insurance/
13 http://www.englishforum.ch/introductions/
14 http://www.englishforum.ch/language-corner/
15 http://www.englishforum.ch/leaving-switzerland/
16 http://www.englishforum.ch/market-place

## Load all data at once, parse it and save it again

Now that we have downloaded our data and save it to localstorage, we need to be transform it from ”dirty” to clean, as we saw during the course.
We will remove some incoherent information, treat null and parse data.

In [61]:
import glob

dfAuthors={}
filenames = [fn for fn in glob.glob("csv/*.csv") 
         if os.path.basename(fn).endswith('Authors.csv')]
for index,filename in enumerate(filenames):
    df = pd.read_csv(filename, header=0,sep='\t',index_col=0)
    dfAuthors[re.sub(r'.csv','',os.path.basename(filename))]=df
print(filenames,dfAuthors.keys())

['csv/Business & entrepreneurAuthors.csv', 'csv/ConcertsAuthors.csv', 'csv/EducationAuthors.csv', 'csv/PermitsvisasgovernmentAuthors.csv', 'csv/Commercial eventsAuthors.csv', 'csv/Travelday tripsfree timeAuthors.csv', 'csv/Daily lifeAuthors.csv', 'csv/EmploymentAuthors.csv', 'csv/TransportationdrivingAuthors.csv', 'csv/Language cornerAuthors.csv', 'csv/Swiss news via The LocalAuthors.csv', 'csv/TVinternettelephoneAuthors.csv', 'csv/OthergeneralAuthors.csv', 'csv/Swiss politicsnewsAuthors.csv', 'csv/IntroductionsAuthors.csv', 'csv/Social eventsAuthors.csv', 'csv/Food and drinkAuthors.csv', 'csv/Leaving SwitzerlandAuthors.csv', 'csv/FinancebankingtaxationAuthors.csv', 'csv/Family mattershealthAuthors.csv', 'csv/InsuranceAuthors.csv', 'csv/Entertainment & diningAuthors.csv', 'csv/Complaints cornerAuthors.csv', 'csv/Pet cornerAuthors.csv', 'csv/Sports  Fitness  Beauty  WellnessAuthors.csv'] dict_keys(['ConcertsAuthors', 'Complaints cornerAuthors', 'InsuranceAuthors', 'Sports  Fitness  Beau

This is how our data initially looks like:

In [51]:
dfAuthors['ConcertsAuthors'].head(3)

,date,exp,groaned,location,post,replies,reputation,since,thanked,thread,user,user_posts,views,topic
0,"06.05.2009, 16:20",Forum Veteran,Groaned at 1 Time in 1 Post,Location: Perthia,Mod Insert: Please . Can't be bothered si...,"1,100",a reputation beyond repute,Join Date: Mar 2006,Thanked 901 Times in 444 Posts,"Switzerland Gigs ""Heads Up""",Yokine,"Posts: 1,233","151,570",Concerts
1,"19.11.2016, 19:45",Newbie,Groaned at 0 Times in 0 Posts,"Location: Lenzburg, Aargau",\r\n\t\t\t\r\n\t\t\tI am living in Kanton Aar...,5,no particular reputation at present,Join Date: Nov 2016,Thanked 0 Times in 0 Posts,Small bars/places with acoustic livemusic?,JHC,Posts: 3,931,Concerts
2,"28.11.2015, 22:48",Newbie,Groaned at 0 Times in 0 Posts,Location: Spain,\r\n\t\t\t\r\n\t\t\tHello there! \r\nI am a...,16,no particular reputation at present,Join Date: Jul 2015,Thanked 0 Times in 0 Posts,Jazz clubs Zurich?,aromeroa,Posts: 6,"3,849",Concerts


### General parsing
All cells are string type. We would like to convert all string numbers to integers (or floats) and for that we need to remove the comas.
We also want to transform dates represented in the same way (column "date" can have date types and text as "today" or "yesterday")
Finally, there is useless text mixed with more important data like in the column "location", in which we can find the word "location:" always repeted per cell.

In [52]:
from datetime import datetime, timedelta
import time

for key in dfAuthors:
    # # Casting views from str to int (- to 0 because is a moved)
    today=datetime.strptime(time.ctime(os.path.getctime('csv/'+key+'.csv')),'%a %b %d %H:%M:%S %Y')
    yesterday=today- timedelta(days=1)
    tdy="%d.%d.%d" %(today.day,today.month,today.year)
    ystr="%d.%d.%d" %(yesterday.day,yesterday.month,yesterday.year)
    
    
    dfAuthors[key]['views']=dfAuthors[key]['views'].apply(lambda x: str(x).replace(',',''))
    dfAuthors[key]['replies']=dfAuthors[key]['replies'].apply(lambda x: str(x).replace(',',''))
    #     dfAuthors[key].user_posts=dfAuthors[key].apply(lambda x: x[9].replace('Posts: ','').replace(',','').replace('-','0') ,axis=1)
    #     [print(key) for x in dfAuthors[key]['since'] if 'Location: Lugano' in x]
    #     dfAuthors[key]['views']=dfAuthors[key]['views'].apply(lambda x: int(x.replace(',','').replace('-','0')))
    #   dfAuthors[key]['replies']=dfAuthors[key]['replies'].apply(lambda x: int(str(x).replace(',','').replace('-','0')))
    dfAuthors[key]['user_posts']=dfAuthors[key]['user_posts'].apply(lambda x: ''.join(re.findall(r'\d+',x)))
    dfAuthors[key]['user']=dfAuthors[key]['user'].apply(lambda x: str(x).replace(str(x)[0],'') if((str(x)[0])=='"') else str(x))
    dfAuthors[key]['date']=dfAuthors[key]['date'].apply(lambda x: str(x))
    dfAuthors[key]['date']=dfAuthors[key]['date'].apply(lambda x: x.replace('Yesterday',ystr))
    dfAuthors[key]['date']=dfAuthors[key]['date'].apply(lambda x: x.replace('Today',tdy))
    #     dfAuthors[key]['date']=[datetime.strptime(x, '%d.%m.%Y, %H:%M')for x in dfAuthors[key]['date'] if (x!='null')]
    dfAuthors[key]['since']=dfAuthors[key]['since'].apply(lambda x: str(x))
    dfAuthors[key]['since']=dfAuthors[key]['since'].apply(lambda x: x.replace('Join Date:',''))
    dfAuthors[key]['since']=dfAuthors[key]['since'].apply(lambda x: x.replace('null',(" Jun 0001")))
    #     dfAuthors[key]['since']=[datetime.strptime(x, ' %b %Y')for x in dfAuthors[key]['since']]
    dfAuthors[key]['location']=dfAuthors[key]['location'].apply(lambda x: x.replace('Location: ',''))
    dfAuthors[key].head()
    # Thanked 418 Times in 115 
    # Groaned at 3 Times in 3 Posts
    dfAuthors[key]['groaned']=dfAuthors[key]['groaned'].apply(lambda x: re.findall(r'\d+',x))
    dfAuthors[key]['thanked']=dfAuthors[key]['thanked'].apply(lambda x: re.findall(r'\d+',x))
    # print(filenames)
dfAuthors[key].head(2)


,date,exp,groaned,location,post,replies,reputation,since,thanked,thread,user,user_posts,views,topic
0,"06.05.2009, 16:20",Forum Veteran,"[1, 1]",Perthia,Mod Insert: Please . Can't be bothered si...,1100,a reputation beyond repute,Mar 2006,"[901, 444]","Switzerland Gigs ""Heads Up""",Yokine,1233,151570,Concerts
1,"19.11.2016, 19:45",Newbie,"[0, 0]","Lenzburg, Aargau",\r\n\t\t\t\r\n\t\t\tI am living in Kanton Aar...,5,no particular reputation at present,Nov 2016,"[0, 0]",Small bars/places with acoustic livemusic?,JHC,3,931,Concerts


Reputation transformation. Basic transformation:

In [53]:
print('Reputation values: ',dfAuthors['ConcertsAuthors']['reputation'].unique())
d = {}
d["a reputation beyond repute"] = 6 # 
d["an excellent reputation"] = 5 # 
d["considered knowledgeable"] = 4 #
d["earned the respect of many"] = 3 #
d["earned some respect"] = 2#
d["made some interesting contributions"] = 1 #
d["no particular reputation at present"] = 0#1
d["slipped a little"] = -1 #0
d["become a little unpopular"] = -2 #
d["annoyed a few people around here"] = -3 #
d["considered a nuisance"] = -4 #
d["considered unworthy"] = -5 #
d["null"] = np.nan


Reputation values:  ['a reputation beyond repute' 'no particular reputation at present'
 'slipped a little' 'considered knowledgeable'
 'made some interesting contributions' 'earned some respect' 'null'
 'become a little unpopular' 'an excellent reputation'
 'considered a nuisance' 'annoyed a few people around here'
 'earned the respect of many' 'm' 'considered unworthy']


In [54]:
uniques=set()
[[uniques.add(user_exp) for user_exp in dfAuthors[key]['reputation'] if user_exp not in list(d.keys())]for key in dfAuthors]
uniques

{'m'}

In [55]:
for key in dfAuthors:
    dfAuthors[key]['reputation']=[d.get(item,item)  for item in dfAuthors[key]['reputation']]
dfAuthors[key].head(2)

,date,exp,groaned,location,post,replies,reputation,since,thanked,thread,user,user_posts,views,topic
0,"06.05.2009, 16:20",Forum Veteran,"[1, 1]",Perthia,Mod Insert: Please . Can't be bothered si...,1100,6,Mar 2006,"[901, 444]","Switzerland Gigs ""Heads Up""",Yokine,1233,151570,Concerts
1,"19.11.2016, 19:45",Newbie,"[0, 0]","Lenzburg, Aargau",\r\n\t\t\t\r\n\t\t\tI am living in Kanton Aar...,5,0,Nov 2016,"[0, 0]",Small bars/places with acoustic livemusic?,JHC,3,931,Concerts


In [56]:
print('Reputation values: ',dfAuthors[key]['exp'].unique())
# Basic exp
d = {}
d["Newbie"] = 0 # 
d["Newbie 1st class"] = 1 #
d["Junior Member"] = 2 #
d["Member"] = 3#
d["Senior Member"] = 4 #
d["Forum Veteran"] = 5#1
d["Forum Legend"] = 6 #0
d["Commercial paid-placement ads"] = 7 # 
d["Banned"] = -1 #
d["null"] = np.nan #

Reputation values:  ['Forum Veteran' 'Newbie' 'Forum Legend' 'Junior Member' 'Member'
 'Newbie 1st class' 'Senior Member' 'null' 'Banned'
 'modified and reprogrammed' 'RIP' 'Moderato espressivo']


There are some special rols:

In [57]:
other_rols=set()
[[other_rols.add(user_exp) for user_exp in dfAuthors[key]['exp'] if user_exp not in list(d.keys())]for key in dfAuthors]
other_rols

{'Moderato espressivo', 'RIP', 'modified and reprogrammed'}

Because they aren't so many of them. We will parse them manually.

In [58]:
# dfAuthors['EducationAuthors'].head()
d["A singular modality"] = 8 # Special rol
d["Moderately Amused"] = 8 # Special rol
d["Only in moderation"] = 8 # Special rol
d["The Architect"] = 8 # Special rol
d["Unbridled Mod"] = 8 # Special rol
d["à la mod"] = 8 #
d["RIP"] = 8 # Special rol
d["Moddy McModface"] = 8 # Special rol
d["Moderato espressivo"] = 8 # Special rol
d["modified and reprogrammed"] = 8 #

# temp=map(d.get, dfAuthors['EducationAuthors']['reputation'], dfAuthors['EducationAuthors']['reputation'])
for key in dfAuthors:
    dfAuthors[key]['exp']=[d.get(item,item)  for item in dfAuthors[key]['exp']]
# dfAuthors['Complaints cornerAuthors'][(dfAuthors['Complaints cornerAuthors']['exp']=='Join Date: Dec 2009')]
print('Exp values: ',dfAuthors[key]['exp'].unique())


Exp values:  [  5.   0.   6.   2.   3.   1.   4.  nan  -1.   8.]


In [59]:
dfAuthors['ConcertsAuthors'].head(2)

,date,exp,groaned,location,post,replies,reputation,since,thanked,thread,user,user_posts,views,topic
0,"06.05.2009, 16:20",5.0,"[1, 1]",Perthia,Mod Insert: Please . Can't be bothered si...,1100,6,Mar 2006,"[901, 444]","Switzerland Gigs ""Heads Up""",Yokine,1233,151570,Concerts
1,"19.11.2016, 19:45",0.0,"[0, 0]","Lenzburg, Aargau",\r\n\t\t\t\r\n\t\t\tI am living in Kanton Aar...,5,0,Nov 2016,"[0, 0]",Small bars/places with acoustic livemusic?,JHC,3,931,Concerts


In [47]:
dfAuthors['Business & entrepreneurAuthors'].head(2)

,topic,thread,user,views,replies,location,date,post,user_posts,since,exp,thanked,groaned,reputation
0,Business & entrepreneur,Posting in this area - guidelines - rules - re...,mark,17624,0,"Zollikon, Switzerland","11.08.2006, 15:44",\r\n\t\t\t\r\n\t\t\tThis area is intended as ...,3070.0,May 2005,8.0,"['3', '3']","['418', '115']",6
1,Business & entrepreneur,Questions about liability and invoicing of Fre...,Nano,139,2,Solothurn,"18.1.2017, 23:08",\r\n\t\t\t\r\n\t\t\tI work though my own GmbH...,2.0,Jan 2017,0.0,"['0', '0']","['0', '0']",0


In [60]:
for key in dfAuthors:
    saveData(dfAuthors[key],(key))